# Parsing the field task excel sheet to extract relevant data

In [99]:
import pandas as pd
import re
import numpy as np

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth',None)
pd.options.mode.chained_assignment = None

In [225]:
df = pd.read_excel('Downloads/change_request_hardware_Jan_to_Nov_2020.xlsx',sheet_name=0)


#Standard Table
std_df = pd.read_excel("D:\Data Viz\Cluster_Details.xlsx")

#### Function to extract the RFC number

In [128]:
def extract_rfc(dataframe):
    rfc = dataframe[['Number']].copy()
    return rfc
    

#### Function to extract the server name

In [404]:
def extract_name(dataframe):
    
    #Select only the rows needed.
    ServerName_df = dataframe[['Short description','Description','Implementation plan']]
    
    #Extract only relevant information and discard the rest in each column
    for col in ServerName_df.columns:
        ServerName_df[col] = ServerName_df[col].str.extract(r'([exEX]\w{2,7}\d{2})',expand=False).str.strip()
        
    #Replacing NaN values with 'x' so that when counting length of characters, we don't get TypeError.
    ServerName_df = ServerName_df.fillna('x')
    
    #Length list will store the column with longest name for server in that row.
    length = []
    machine_name = []
    
    for row in range(ServerName_df.shape[0]):
        for col in ServerName_df.loc[row,]:
            length.append(len(col))
        longest = np.argmax(length)
        machine_name.append(ServerName_df.iloc[row,longest]) #Append the column value with the longest string to our list.
        #Reset the length list for the next row
        length = []
    
    #Convert all to lowercase
    machine_name = map(str.lower,machine_name)        
        
    return pd.DataFrame(machine_name,columns = ['Machine name'])


#### Function to extract Data center

In [219]:
def extract_data_center(dataframe):
    Datacenter = dataframe['Implementer Group'].str.extract(r'(TPC|NJ|TN2)')
    Datacenter = Datacenter.replace('NJ','CNJ')
    Datacenter.columns = ['Datacenter']
    return Datacenter    

#### Function to extract Date of Event

In [133]:
def extract_date(dataframe):
    Start_time = dataframe[['Planned start date']]
    return Start_time

#### Function to extract Faulty component

In [221]:
def extract_faulty(dataframe):
    #Define a pattern to extract. The most commonly observed failures are included to search in the text.
    pattern = re.compile(r'(DIMM|disk|pcie|eth|processor|battery|fan module|cable|ilom|motherboard|sensor|IB switch|power supply|upgrading|PDU|PSU)',re.I)
    
    #str.title capitalizes first letter of each word.
    faulty_component = dataframe['Description'].str.extract(pattern,expand=False).str.strip().str.title()
    
    #Replace some non standard data entries if found.
    faulty_component = faulty_component.replace(['Upgrading','Pdu','Psu'],['Memory upgrade','Power Supply','Power Supply'])
    faulty_component = pd.DataFrame(faulty_component)
    faulty_component.columns = ['Faulty component']
    
    return faulty_component

#### Function to extract cluster names

In [433]:
def extract_cluster(dataframe):
    
    Machine_name = extract_name(df)
    
    #Pattern to extract
    pattern = re.compile(r'(.{5})',re.I)
    
    cluster = Machine_name['Machine name'].str.extract(pattern,expand=False).str.strip().str.lower()
    cluster = pd.DataFrame(cluster)
    cluster.columns = ['Cluster']
    
    return cluster

#### Function to extract Node type

In [427]:
def extract_node_type(dataframe):
    
    Machine_name = extract_name(df)
    
    conditions = [Machine_name['Machine name'].str.contains(r'(db|DB)',regex=True),
            Machine_name['Machine name'].str.contains(r'(cel|CEL)',regex=True),
            Machine_name['Machine name'].str.contains(r'(ib|IB)',regex=True),
            Machine_name['Machine name'].str.contains(r'(zfs|ZFS)',regex=True),
            Machine_name['Machine name'].str.contains(r'(zdm|ZDM)',regex=True)]
    
    values = ['Database','Cell','IB-Switch','ZFS-Storage','ZDLRA-db']
    
    node_type = np.select(conditions,values)
    node_type = pd.DataFrame(node_type)
    node_type.columns = ['Node type']
    
    return node_type   
    

#### Function to extract Machine type

In [430]:
def extract_machine_type(dataframe):
    
    Node_type = extract_node_type(df)
    
    conditions = [Node_type['Node type'].str.contains(r'^ZFS',regex=True),
             Node_type['Node type'].str.contains(r'^ZDLRA',regex=True),
             Node_type['Node type'].str.contains(r'^Database|^Cell|^IB',regex=True),
             ]
    
    values = ['ZFS','ZDLRA','Exadata']
    
    machine_type = np.select(conditions,values)
    machine_type = pd.DataFrame(machine_type)
    machine_type.columns = ['Machine type']
    
    return machine_type    
    

#### Function to extract Machine version

In [345]:
def extract_version(dataframe):
    
    version_pattern = re.compile(r'(X\d-\d)',re.I)

    machine_version = dataframe['Description'].str.extract(version_pattern,expand=False).str.strip()
    
    #Fill na if other column has any version information
    machine_version.fillna(dataframe['Description'].str.extract(version_pattern,expand=False).str.strip(),inplace = True)
    
    machine_version = pd.DataFrame(machine_version)
    machine_version.columns = ['Machine version']
    
    return machine_version

#### Function to extract Downtime

In [424]:
def extract_downtime(dataframe):
    
    Faulty_component = extract_faulty(df)
    
    downtime = np.where(Faulty_component['Faulty component'].str.contains(r'(Dimm|Processor|Ib Switch|Motherboard|Ilom|NodeDown|Battery)'),\
                             'Yes','No')
    
    downtime = pd.DataFrame(downtime)
    downtime.columns = ['Downtime']
    
    return downtime
    

#### Function to extract GF vs BF

In [418]:
def extract_region(dataframe):
    
    Machine_name = extract_name(df)
    
    region = np.where(Machine_name['Machine name'].str.contains(r'(xt0|xc0|xlmd|xlmc|elpd|elpc)',regex=True),'BF','GF')
    region = pd.DataFrame(region)
    region.columns = ['Region']
    
    return region
    

#### Function to extract the missing versions

In [465]:
#Try to extract version from the std lookup table.
def missing_versions(dataframe1,std_dataframe):
    x = pd.concat([extract_cluster(dataframe1),extract_version(dataframe1)],axis = 1)
    y = std_dataframe[['Cluster']]
    
    new_df = ms_df.merge(std_df,on = 'Cluster',how = 'left',indicator=True)
    
    new_df['Machine version_x'].fillna(new_df['Machine version_y'],inplace=True)
    
    mv = pd.DataFrame(new_df['Machine version_x'])
    mv.columns = ['Machine version']
    
    return mv
    

#### Function to extract Action time

In [482]:
def extract_action_time(dataframe):
    difference = (df['Planned start date'] - df['Created'])/np.timedelta64(1,'D')
    difference = round(difference,0)
    difference.astype(int)
    
    difference = pd.DataFrame(difference)
    difference.columns = ['Action time']
    
    return difference
    

#### Create Final table

In [1]:
def create_table(dataframe):
    table = pd.concat([extract_rfc(dataframe),extract_machine_type(dataframe),extract_node_type(dataframe),\
                       extract_name(dataframe),extract_cluster(dataframe),missing_versions(dataframe,std_dataframe),\
                       extract_faulty(dataframe),extract_date(dataframe),extract_data_center(dataframe),\
                       extract_region(dataframe),extract_downtime(dataframe),extract_action_time(dataframe)],axis = 1)
    
    return table

In [484]:
final_df = create_table(df)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [486]:
final_df.to_excel('D:\Data Viz\FuncOut.xlsx',index=False)

- extract_rfc
- extract_machine_type
- extract_node_type
- extract_name
- extract_cluster
- extract_version
- extract_faulty
- extract_date
- extract_data_center
- extract_region
- extract_downtime
- missing_versions
- extract_action_time
- create_table



